In [2]:
# volume_gui.py
# Windows only. Dependencies: pycaw, comtypes
# pip install pycaw comtypes

import sys
import tkinter as tk
from tkinter import messagebox

# Ensure running on Windows
if sys.platform != "win32":
    message = "This program uses Windows Core Audio via PyCAW and runs only on Windows."
    raise SystemExit(message)

try:
    from comtypes import CLSCTX_ALL, cast
    from ctypes import POINTER
    from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
except Exception as e:
    raise SystemExit("Failed to import required modules. Install with: pip install pycaw comtypes\n\nError: " + str(e))


class VolumeGUI(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("System Volume Controller (PyCAW + Tkinter)")
        self.geometry("360x220")
        self.resizable(False, False)

        # Try to get endpoint volume on startup
        try:
            self.vol = self.get_endpoint_volume()
        except Exception as e:
            messagebox.showerror("Error", f"Could not access audio endpoint.\n\n{e}")
            self.vol = None

        # UI elements
        self.create_widgets()

        # Start polling to update UI with system volume/mute changes
        self.poll_interval_ms = 500  # update every 500 ms
        self.after(self.poll_interval_ms, self.poll_system_volume)

    def create_widgets(self):
        pad = 8

        # Status label
        self.status_var = tk.StringVar(value="Status: unknown")
        status_lbl = tk.Label(self, textvariable=self.status_var, anchor="w")
        status_lbl.pack(fill="x", padx=pad, pady=(pad, 0))

        # Slider (0-100)
        self.slider = tk.Scale(self, from_=0, to=100, orient="horizontal", length=300,
                               command=self.on_slider_move)
        self.slider.pack(padx=pad, pady=(6, 0))

        # Bind release to set final value (avoid flooding calls while dragging)
        self.slider.bind("<ButtonRelease-1>", self.on_slider_release)

        # Buttons frame
        btn_frame = tk.Frame(self)
        btn_frame.pack(pady=12)

        inc_btn = tk.Button(btn_frame, text="Increase 5%", width=12, command=lambda: self.change_volume(0.05))
        inc_btn.grid(row=0, column=0, padx=6)

        dec_btn = tk.Button(btn_frame, text="Decrease 5%", width=12, command=lambda: self.change_volume(-0.05))
        dec_btn.grid(row=0, column=1, padx=6)

        mute_btn = tk.Button(btn_frame, text="Toggle Mute", width=12, command=self.toggle_mute)
        mute_btn.grid(row=1, column=0, padx=6, pady=6)

        refresh_btn = tk.Button(btn_frame, text="Refresh Endpoint", width=12, command=self.refresh_endpoint)
        refresh_btn.grid(row=1, column=1, padx=6, pady=6)

        # Quit button
        quit_btn = tk.Button(self, text="Quit", command=self.destroy)
        quit_btn.pack(side="bottom", pady=(0, pad))

        # Disable UI if endpoint not available
        if self.vol is None:
            self.disable_ui()
        else:
            # Initialize UI values from system
            self.update_ui_from_system()

    def disable_ui(self):
        self.slider.config(state="disabled")
        for child in self.winfo_children():
            if isinstance(child, tk.Button):
                child.config(state="disabled")
        self.status_var.set("Status: audio endpoint not available")

    def enable_ui(self):
        self.slider.config(state="normal")
        for child in self.winfo_children():
            if isinstance(child, tk.Button):
                child.config(state="normal")

    def get_endpoint_volume(self):
        """
        Acquire the IAudioEndpointVolume interface and cast to a typed pointer.
        """
        devices = AudioUtilities.GetSpeakers()  # COM object for default speakers
        interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)  # raw COM pointer
        volume = cast(interface, POINTER(IAudioEndpointVolume))  # typed pointer for Python
        return volume

    def refresh_endpoint(self):
        try:
            self.vol = self.get_endpoint_volume()
            if self.vol:
                self.enable_ui()
                self.update_ui_from_system()
                messagebox.showinfo("Refreshed", "Audio endpoint refreshed.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to refresh audio endpoint:\n\n{e}")
            self.vol = None
            self.disable_ui()

    # System <-> UI helpers
    def get_volume_scalar(self):
        """Return 0.0 - 1.0 float or None if no endpoint."""
        if not self.vol:
            return None
        try:
            return float(self.vol.GetMasterVolumeLevelScalar())
        except Exception:
            return None

    def set_volume_scalar(self, value):
        """Set master volume scalar 0.0 - 1.0."""
        if not self.vol:
            return
        value = max(0.0, min(1.0, float(value)))
        try:
            self.vol.SetMasterVolumeLevelScalar(value, None)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to set volume:\n\n{e}")

    def get_mute(self):
        if not self.vol:
            return None
        try:
            return bool(self.vol.GetMute())
        except Exception:
            return None

    def set_mute(self, mute_bool: bool):
        if not self.vol:
            return
        try:
            self.vol.SetMute(int(bool(mute_bool)), None)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to set mute:\n\n{e}")

    # UI actions
    def change_volume(self, delta):
        cur = self.get_volume_scalar()
        if cur is None:
            return
        new = max(0.0, min(1.0, cur + delta))
        self.set_volume_scalar(new)
        self.update_ui_from_system()

    def toggle_mute(self):
        cur = self.get_mute()
        if cur is None:
            return
        self.set_mute(not cur)
        self.update_ui_from_system()

    def update_ui_from_system(self):
        """Read system volume/mute and reflect in UI."""
        v = self.get_volume_scalar()
        m = self.get_mute()
        if v is None or m is None:
            self.status_var.set("Status: audio endpoint not available")
            return
        pct = int(round(v * 100))
        self.slider.set(pct)
        self.status_var.set(f"Volume: {pct}%   Muted: {m}")

    # Slider callbacks
    def on_slider_move(self, val):
        # Optional: show prospective value while dragging
        try:
            pct = int(float(val))
            self.status_var.set(f"Volume (moving): {pct}%")
        except Exception:
            pass

    def on_slider_release(self, event):
        # Set actual volume when user releases mouse
        pct = self.slider.get()
        self.set_volume_scalar(pct / 100.0)
        self.update_ui_from_system()

    def poll_system_volume(self):
        """Periodic polling: keep UI in sync with system changes."""
        if self.vol:
            try:
                self.update_ui_from_system()
            except Exception:
                # ignore transient errors
                pass
        self.after(self.poll_interval_ms, self.poll_system_volume)


if __name__ == "__main__":
    app = VolumeGUI()
    app.mainloop()
